# Main

### Prerequisites

**Imports**

In [ ]:
import os
import sys
import warnings
import numpy as np
import geopandas as gpd
from shapely.geometry import Polygon
import matplotlib.pyplot as plt

**Fix directories, define path variables**

In [ ]:
# make sure notebook is ran from src
cwd = os.getcwd()
if not cwd.split('/')[-1] == 'src':
    try:
        os.chdir('src')
    except FileNotFoundError:
        print('Error: please run from src dir or project root')
        sys.exit(1)

# define paths
data_path = '../data/'
csv_path = data_path + 'csv/'
shp_path = data_path + 'shapefiles/'
plot_path = '../plots/'
if not os.path.exists(plot_path):
    os.mkdir(plot_path)

---

### Read data

In [ ]:
# these are just manhattan gdfs, with correct projection
gdf_streets: gpd.GeoDataFrame = gpd.read_file(shp_path + 'streets/M_streets.shp')
gdf_traffic: gpd.GeoDataFrame = gpd.read_file(shp_path + 'traffic/M_traffic.shp')
gdf_trees: gpd.GeoDataFrame = gpd.read_file(shp_path + 'trees/M_trees.shp')

Now we create a grid.

In [ ]:
# define a bounding box around Manhattan
xmin, ymin, xmax, ymax = gdf_streets.total_bounds
bbox = Polygon([(xmin, ymin), (xmin, ymax), (xmax, ymax), (xmax, ymin)])

# define granularity of grid
l, w = .005, .005

# create a grid of polygons
cols = list(np.arange(xmin, xmax+w, w))
rows = list(np.arange(ymin, ymax+l, l))
polygons = []
for x in cols[:-1]:
    for y in rows[:-1]:
        polygons.append(Polygon([(x, y), (x, y+l), (x+w, y+l), (x+w, y), (x, y)]))

# create a gdf from the grid
grid = gpd.GeoDataFrame({'geometry': polygons})
grid.crs = 4326

# remove grid cells that do not intersect with any street
size_before = grid.shape[0]
grid = grid[grid.intersects(gdf_streets.unary_union)]
print(f'Grid cells trimmed from {grid.shape[0]} to {size_before}')

In [ ]:
def add_grid_plot(ax, gdf_streets, gdf_traffic, gdf_trees, grid, title, lims):
    gdf_streets.plot(ax=ax, color='grey', alpha=0.5, zorder=1)
    grid.plot(ax=ax, color='none', edgecolor='black', linewidth=1, zorder=2)
    gdf_trees.plot(ax=ax, markersize=0.05, color='green', alpha=0.1, zorder=3)
    gdf_traffic.plot(ax=ax, column='Avg_Vol', markersize=5, cmap='Reds', zorder=3)
    # set lims
    ax.set_xlim(lims[0], lims[2])
    ax.set_ylim(lims[1], lims[3])
    ax.set_axis_off()
    ax.set_title(title)

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(10, 10))

lims = xmin-w, ymin-l, xmax+w, ymax+l  # comes from gdf_streets
add_grid_plot(ax1, gdf_streets, gdf_traffic, gdf_trees, grid, 'before', lims)

grid_size_before, streets_size_before = grid.shape[0], gdf_streets.shape[0]

# remove the island cells
lowest_x = -74.03676859535565  # = lowest x value of the main land
# we're using geographic CRS, but we know this coordinate is correct
with warnings.catch_warnings():
    warnings.simplefilter('ignore')
    grid = grid[grid.geometry.centroid.x > lowest_x]
# remove the streets that were inside these cells
gdf_streets = gdf_streets[gdf_streets.geometry.intersects(grid.unary_union)]

print(f'Grid cells trimmed from {grid_size_before} to {grid.shape[0]}')
print(f'Streets trimmed from {streets_size_before} to {gdf_streets.shape[0]}')

add_grid_plot(ax2, gdf_streets, gdf_traffic, gdf_trees, grid, 'after', lims)
fig.tight_layout()

Now we can assign the relevant data to each grid cell.

In [ ]:
# we would get a SettingWithCopyWarning, but it's not a problem
with warnings.catch_warnings():
    warnings.simplefilter('ignore')
    grid['trees'] = grid.geometry.apply(lambda x: gdf_trees[gdf_trees.geometry.within(x)].shape[0])
    grid['traffic'] = grid.geometry.apply(lambda x: gdf_traffic[gdf_traffic.geometry.within(x)].Avg_Vol.mean())

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(10, 10))
for ax, col, cmap in zip([ax1, ax2], ['trees', 'traffic'], ['Greens', 'Reds']):
    grid.plot(ax=ax, column=col, cmap=cmap, legend=True, zorder=1, legend_kwds={'orientation': 'horizontal', 'shrink': 0.5})
    gdf_streets.plot(ax=ax, color='grey', alpha=0.1, zorder=2)
    ax.set_axis_off()
    ax.set_title(col.capitalize())
fig.tight_layout()
fig.savefig(plot_path + 'grid_raw.png', dpi=500)

We'll save the raw grid data to a shapefile again.

In [ ]:
if not os.path.exists((target := shp_path + 'grid/')):
    os.mkdir(target)
grid.to_file(target + 'M_grid.shp')